# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 27 2022 9:19AM,253801,12,HH-38953,Hush Hush,Released
1,Dec 27 2022 9:19AM,253801,12,HH-38955,Hush Hush,Released
2,Dec 27 2022 9:19AM,253801,12,HH-38956,Hush Hush,Released
3,Dec 27 2022 9:19AM,253801,12,HH-38957,Hush Hush,Released
4,Dec 27 2022 9:19AM,253801,12,HH-38958,Hush Hush,Released
5,Dec 27 2022 9:19AM,253801,12,HH-38959,Hush Hush,Released
6,Dec 27 2022 9:19AM,253801,12,HH-38960,Hush Hush,Released
7,Dec 27 2022 9:19AM,253801,12,HH-38961,Hush Hush,Released
8,Dec 27 2022 9:19AM,253801,12,HH-38962,Hush Hush,Released
9,Dec 27 2022 9:19AM,253801,12,HH-38963,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,253801,Released,38
14,253802,Released,1
15,253803,Released,2
16,253804,Released,1
17,253805,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253801,NaN,38.0
253802,NaN,1.0
253803,NaN,2.0
253804,NaN,1.0
253805,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253783,0.0,1.0
253784,13.0,16.0
253785,1.0,0.0
253788,0.0,1.0
253789,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253783,0,1
253784,13,16
253785,1,0
253788,0,1
253789,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253783,0,1
1,253784,13,16
2,253785,1,0
3,253788,0,1
4,253789,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253783,,1
1,253784,13,16
2,253785,1,
3,253788,,1
4,253789,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 27 2022 9:19AM,253801,12,Hush Hush
38,Dec 27 2022 9:17AM,253805,19,ACG North America LLC
39,Dec 27 2022 9:13AM,253804,10,"Senseonics, Incorporated"
40,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc."
42,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation
82,Dec 27 2022 9:10AM,253802,10,"Methapharm, Inc."
83,Dec 27 2022 9:03AM,253799,10,Bio-PRF
84,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation
145,Dec 27 2022 9:02AM,253797,12,Hush Hush
146,Dec 27 2022 9:01AM,253796,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 27 2022 9:19AM,253801,12,Hush Hush,,38
1,Dec 27 2022 9:17AM,253805,19,ACG North America LLC,,1
2,Dec 27 2022 9:13AM,253804,10,"Senseonics, Incorporated",,1
3,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",,2
4,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,,40
5,Dec 27 2022 9:10AM,253802,10,"Methapharm, Inc.",,1
6,Dec 27 2022 9:03AM,253799,10,Bio-PRF,,1
7,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation,,61
8,Dec 27 2022 9:02AM,253797,12,Hush Hush,,1
9,Dec 27 2022 9:01AM,253796,10,Bio-PRF,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:19AM,253801,12,Hush Hush,38,
1,Dec 27 2022 9:17AM,253805,19,ACG North America LLC,1,
2,Dec 27 2022 9:13AM,253804,10,"Senseonics, Incorporated",1,
3,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",2,
4,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,40,
5,Dec 27 2022 9:10AM,253802,10,"Methapharm, Inc.",1,
6,Dec 27 2022 9:03AM,253799,10,Bio-PRF,1,
7,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation,61,
8,Dec 27 2022 9:02AM,253797,12,Hush Hush,1,
9,Dec 27 2022 9:01AM,253796,10,Bio-PRF,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:19AM,253801,12,Hush Hush,38,
1,Dec 27 2022 9:17AM,253805,19,ACG North America LLC,1,
2,Dec 27 2022 9:13AM,253804,10,"Senseonics, Incorporated",1,
3,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",2,
4,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,40,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:19AM,253801,12,Hush Hush,38.0,NaN
1,Dec 27 2022 9:17AM,253805,19,ACG North America LLC,1.0,NaN
2,Dec 27 2022 9:13AM,253804,10,"Senseonics, Incorporated",1.0,NaN
3,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",2.0,NaN
4,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,40.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:19AM,253801,12,Hush Hush,38.0,0.0
1,Dec 27 2022 9:17AM,253805,19,ACG North America LLC,1.0,0.0
2,Dec 27 2022 9:13AM,253804,10,"Senseonics, Incorporated",1.0,0.0
3,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",2.0,0.0
4,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,40.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2791758,132.0,0.0
12,507598,39.0,0.0
19,761373,2.0,1.0
20,253784,16.0,13.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2791758,132.0,0.0
1,12,507598,39.0,0.0
2,19,761373,2.0,1.0
3,20,253784,16.0,13.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,132.0,0.0
1,12,39.0,0.0
2,19,2.0,1.0
3,20,16.0,13.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,132.0
1,12,Released,39.0
2,19,Released,2.0
3,20,Released,16.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20
Status,,,,
Executing,0.0,0.0,1.0,13.0
Released,132.0,39.0,2.0,16.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20
0,Executing,0.0,0.0,1.0,13.0
1,Released,132.0,39.0,2.0,16.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20
0,Executing,0.0,0.0,1.0,13.0
1,Released,132.0,39.0,2.0,16.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()